In [1]:
import numpy as np
import cv2 as cv

# Lecture de l'image
img = cv.imread('/Users/mac/Desktop/MST AISD/S3/mutimedia mining/multimedia-mining-indexing labs/lab8/Images/a2.jpg')  # Remplacez par le chemin de l'image
if img is None:
    raise FileNotFoundError("L'image spécifiée est introuvable !")

# Conversion en niveaux de gris
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

# Variables globales pour le dessin
drawing = False  # True si on dessine
mode = True  # True pour les objets, False pour le fond
ix, iy = -1, -1  # Coordonnées initiales

# Préparer les marqueurs
markers = np.zeros(gray.shape, dtype=np.int32)

# Fonction de dessin
def draw_circle(event, x, y, flags, param):
    global ix, iy, drawing, mode
    if event == cv.EVENT_LBUTTONDOWN:  # Clic gauche enfoncé
        drawing = True
        ix, iy = x, y
    elif event == cv.EVENT_MOUSEMOVE:  # Mouvement de la souris
        if drawing:
            if mode:
                cv.circle(markers, (x, y), 5, (2), -1)  # Étiquette 2 pour les objets
            else:
                cv.circle(markers, (x, y), 5, (1), -1)  # Étiquette 1 pour le fond
    elif event == cv.EVENT_LBUTTONUP:  # Clic gauche relâché
        drawing = False

# Créer une fenêtre et l'associer à la fonction de dessin
cv.namedWindow('Dessin')
cv.setMouseCallback('Dessin', draw_circle)

while True:
    # Copie de l'image pour afficher les marqueurs
    img_copy = img.copy()
    img_copy[markers == 2] = [0, 255, 0]  # Zones objets en vert
    img_copy[markers == 1] = [0, 0, 255]  # Zones fond en rouge
    cv.imshow('Dessin', img_copy)
    k = cv.waitKey(1) & 0xFF
    if k == 27:  # Échap pour quitter
        break
    elif k == ord('m'):  # 'm' pour changer de mode
        mode = not mode

cv.destroyAllWindows()

# Vérifier s'il y a des marqueurs avant de continuer
if not np.any(markers == 2):
    raise ValueError("Aucun marqueur d'objet n'a été défini. Dessinez des marqueurs avant de quitter !")

# Appliquer l'algorithme des lignes de partage des eaux
markers[markers == 1] = 1  # Fond
markers[markers == 2] = 2  # Objets
markers[markers == 0] = 0  # Zones neutres

# Application des lignes de partage des eaux
markers_watershed = cv.watershed(img, markers)

# Création de l'image résultat
result = img.copy()
result[markers_watershed == -1] = [255, 255, 255]  # Lignes de partage en blanc
for label in range(2, markers.max() + 1):  # Coloration aléatoire pour chaque segment
    result[markers_watershed == label] = [np.random.randint(0, 256), 
                                          np.random.randint(0, 256), 
                                          np.random.randint(0, 256)]

# Affichage du résultat
while True:
    cv.imshow('Segmentation', result)
    k = cv.waitKey(33)  # 33ms delay for ~30fps
    if k == 27:  # Échap pour quitter
        break

cv.destroyAllWindows()

2024-11-27 09:33:51.243 python[73315:2377831] +[IMKClient subclass]: chose IMKClient_Modern
2024-11-27 09:33:51.243 python[73315:2377831] +[IMKInputSession subclass]: chose IMKInputSession_Modern


: 